In [1]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
import sklearn
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV


In [2]:
df_anime = pd.read_csv("../data/df_anime_export.csv")
df_profile = pd.read_csv("../data/df_profile_export.csv")
df_review = pd.read_csv("../data/df_review_export.csv")
df_an_pro_rev = pd.read_csv("../data/merged_rev_df_export.csv")
df_an_pro = pd.read_csv("../data/merged_df_export.csv")

In [3]:
df_comb_1 = df_anime.merge(df_review, on = 'uid')
df_comb_1

,Unnamed: 0_x,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,Unnamed: 0_y,profile,review_score,review_score_indepth,text_preprocessed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",2,skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen..."
1,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",25036,Tactician,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",continuing flow review haikyuu ! ! ... 'd say ...
2,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",25037,CallMeDette,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",second season haikyuu ! ! truly amazing . love...
3,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",25038,jason200911,7,"{'Overall': '7', 'Story': '7', 'Animation': '7...","sorry 's nowhere good eyeshield le emotion , s..."
4,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",25039,comiclove,8,"{'Overall': '8', 'Story': '7', 'Animation': '7...","second season smash-hit sport anime , haikyuu ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192107,19005,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",181374,SanaeK10,2,"{'Overall': '2', 'Story': '4', 'Animation': '1...",`` went expecting nothing still got disappoint...
192108,19006,16934,Chuunibyou demo Koi ga Shitai!: Kirameki no......,Although Yuuta Togashi and Rikka Takanashi hav...,"['Comedy', 'Drama', 'Romance', 'School', 'Slic...","Jun 19, 2013",1.0,128400,883,1533.0,7.56,although yuuta togashi rikka takanashi started...,181609,Seidai,6,"{'Overall': '6', 'Story': '6', 'Animation': '8...",short episode continuation first season chuuni...
192109,19006,16934,Chuunibyou demo Koi ga Shitai!: Kirameki no......,Although Yuuta Togashi and Rikka Takanashi hav...,"['Comedy', 'Drama', 'Romance', 'School', 'Slic...","Jun 19, 2013",1.0,128400,883,1533.0,7.56,although yuuta togashi rikka takanashi started...,181610,Drevorn,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...","nice little extra episode . slowing finale , c..."
192110,19006,16934,Chuunibyou demo Koi ga Shitai!: Kirameki no......,Although Yuuta Togashi and Rikka Takanashi hav...,"['Comedy', 'Drama', 'Romance', 'School', 'Slic...","Jun 19, 2013",1.0,128400,883,1533.0,7.56,although yuuta togashi rikka takanashi started...,181611,LCSeixas,6,"{'Overall': '6', 'Story': '6', 'Animation': '8...","hey , another episode chuu2 , n't cool ? yeah ..."


In [4]:
df_comb_1.drop(columns = 'Unnamed: 0_x', inplace = True)
df_comb_1.drop(columns = 'Unnamed: 0_y', inplace = True)

In [5]:
df_rev_anime = df_comb_1

In [7]:
df_rev_anime_prof = df_rev_anime.merge(df_profile, on = 'profile')
df_rev_anime_prof

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,profile,review_score,review_score_indepth,text_preprocessed,Unnamed: 0,gender,birthday,favorites_anime
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
1,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",28363,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
2,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",31977,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
3,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",80405,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
4,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479833,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",sensei69,5,"{'Overall': '5', 'Story': '5', 'Animation': '9...",* * * going spoiler free review ... * * * okay...,77347,Male,"Jul 16, 1999","['16774', '23605', '30', '5114', '9253', '437'..."
479834,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ssjtk,8,"{'Overall': '8', 'Story': '8', 'Animation': '1...","n't listen critic , usually idea talking . mov...",77348,NaN,"Jul 14, 1996","['6045', '15583', '6547', '14289', '1535', '31..."
479835,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",Ground_zero,9,"{'Overall': '9', 'Story': '7', 'Animation': '1...",final fantasy fan since old day although 's fa...,77349,Male,"Apr 7, 1982","['43', '5114', '11061', '1', '543', '33082']"
479836,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ClawViper,7,"{'Overall': '7', 'Story': '6', 'Animation': '1...",final fantasy fan following development final ...,77350,NaN,Jul 19,"['13125', '38524', '14397']"


In [8]:
df_rev_anime_prof.drop_duplicates(subset=['profile', 'uid'], inplace=True)

In [9]:
df_rev_anime_prof

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed,profile,review_score,review_score_indepth,text_preprocessed,Unnamed: 0,gender,birthday,favorites_anime
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ...",skrn,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...","art : great , especially action critical momen...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
8,32935,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,"After the victory against Aoba Jousai High, Ka...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 8, 2016 to Dec 10, 2016",10.0,395375,223,16.0,8.94,"victory aoba jousai high , karasuno high schoo...",skrn,8,"{'Overall': '8', 'Story': '9', 'Animation': '9...","oh boy , finally stepped game good-but-not-gre...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
16,30276,One Punch Man,The seemingly ordinary and unimpressive Saitam...,"['Action', 'Sci-Fi', 'Comedy', 'Parody', 'Supe...","Oct 5, 2015 to Dec 21, 2015",12.0,1475219,5,50.0,8.68,seemingly ordinary unimpressive saitama rather...,skrn,7,"{'Overall': '7', 'Story': '6', 'Animation': '1...",one punch man reminds successful hollywood act...,2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
24,4107,Tengen Toppa Gurren Lagann Movie 1: Gurren-hen,Animation studio Gainax presented a website fo...,"['Action', 'Comedy', 'Mecha']","Sep 6, 2008",1.0,112211,1003,349.0,8.18,animation studio gainax presented website rele...,skrn,7,"{'Overall': '7', 'Story': '6', 'Animation': '9...","movie recap first half tv series , % r use mat...",2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
32,4081,Natsume Yuujinchou,"While most fifteen-year-old boys, in one way o...","['Slice of Life', 'Demons', 'Supernatural', 'D...","Jul 8, 2008 to Sep 30, 2008",13.0,344207,276,176.0,8.37,"fifteen-year-old boy , one way another , harbo...",skrn,4,"{'Overall': '4', 'Story': '3', 'Animation': '5...",friend strongly recommended series insisted ne...,2,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479833,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",sensei69,5,"{'Overall': '5', 'Story': '5', 'Animation': '9...",* * * going spoiler free review ... * * * okay...,77347,Male,"Jul 16, 1999","['16774', '23605', '30', '5114', '9253', '437'..."
479834,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ssjtk,8,"{'Overall': '8', 'Story': '8', 'Animation': '1...","n't listen critic , usually idea talking . mov...",77348,NaN,"Jul 14, 1996","['6045', '15583', '6547', '14289', '1535', '31..."
479835,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",Ground_zero,9,"{'Overall': '9', 'Story': '7', 'Animation': '1...",final fantasy fan since old day although 's fa...,77349,Male,"Apr 7, 1982","['43', '5114', '11061', '1', '543', '33082']"
479836,33082,Kingsglaive: Final Fantasy XV,"For years, the Niflheim Empire and the kingdom...",['Action'],"Jul 9, 2016",1.0,41077,2154,1544.0,7.56,"year , niflheim empire kingdom lucis war . emp...",ClawViper,7,"{'Overall': '7', 'Story': '6', 'Animation': '1...",final fantasy fan following development final ...,77350,NaN,Jul 19,"['13125', '38524', '14397']"


In [10]:
df_examine = df_rev_anime_prof[["uid","title","profile","review_score","text_preprocessed"]]

In [11]:
df_examine

,uid,title,profile,review_score,text_preprocessed
0,28891,Haikyuu!! Second Season,skrn,7,"art : great , especially action critical momen..."
8,32935,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,skrn,8,"oh boy , finally stepped game good-but-not-gre..."
16,30276,One Punch Man,skrn,7,one punch man reminds successful hollywood act...
24,4107,Tengen Toppa Gurren Lagann Movie 1: Gurren-hen,skrn,7,"movie recap first half tv series , % r use mat..."
32,4081,Natsume Yuujinchou,skrn,4,friend strongly recommended series insisted ne...
...,...,...,...,...,...
479833,33082,Kingsglaive: Final Fantasy XV,sensei69,5,* * * going spoiler free review ... * * * okay...
479834,33082,Kingsglaive: Final Fantasy XV,ssjtk,8,"n't listen critic , usually idea talking . mov..."
479835,33082,Kingsglaive: Final Fantasy XV,Ground_zero,9,final fantasy fan since old day although 's fa...
479836,33082,Kingsglaive: Final Fantasy XV,ClawViper,7,final fantasy fan following development final ...


In [16]:
df_rev_anime_prof['review_score'].value_counts()

9     24543
10    24520
8     23727
7     19064
6     12446
5      8548
4      5880
3      5726
2      3149
1      2916
Name: review_score, dtype: int64

In [15]:
df_rev_anime_prof['review_score'] = df_rev_anime_prof['review_score'].replace(0,10)
df_rev_anime_prof['review_score'] = df_rev_anime_prof['review_score'].replace(11,10)


In [17]:
df_rev_anime_prof.to_csv('df_prof_rev_anime_clean.csv')

## Creating initial collab recommender

In [27]:
reader = Reader(rating_scale=(1, 10))

In [28]:
df_touse = df_rev_anime_prof[["profile","uid","review_score"]]

In [29]:
df = Dataset.load_from_df(df_touse, reader)

In [31]:
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [35]:
trainingSet = df.build_full_trainset()

In [36]:
df_touse.head(1)

,profile,uid,review_score
0,skrn,28891,7


In [38]:
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [70]:
prediction = algo.predict('Mortrialus', 1575)
prediction.est

3.592809978208985

## Some tuning

In [76]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4],
    "user_based": [True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2)
gs.fit(df)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
2.3432956621026055
{'sim_options': {'name': 'cosine', 'min_support': 4, 'user_based': True}}


In [77]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2)
gs.fit(df)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
2.2047693584415655
{'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


## Alternative way to collab filter

In [80]:
df_review.drop(columns = "Unnamed: 0", inplace = True)

In [82]:
df_review.head(1)

,profile,uid,review_score,review_score_indepth,text_preprocessed
0,DesolatePsyche,34096,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",first thing first . `` review '' system explai...


In [81]:
df_review.sort_values('profile')

,profile,uid,review_score,review_score_indepth,text_preprocessed
16382,-----noname-----,2025,4,"{'Overall': '4', 'Story': '6', 'Animation': '3...",like anime ended dropping episode therefore re...
108940,-----noname-----,2025,4,"{'Overall': '4', 'Story': '6', 'Animation': '3...",like anime ended dropping episode therefore re...
149194,-----noname-----,18441,2,"{'Overall': '2', 'Story': '1', 'Animation': '6...",giant cliffhanger end season . chose sum ovum ...
38672,---SnowFlake---,1535,6,"{'Overall': '6', 'Story': '4', 'Animation': '6...","opinion would master piece , felt like rushed ..."
168662,---SnowFlake---,1535,6,"{'Overall': '6', 'Story': '4', 'Animation': '6...","opinion would master piece , felt like rushed ..."
...,...,...,...,...,...
22258,zzzjynne,6675,7,"{'Overall': '7', 'Story': '4', 'Animation': '1...","amazing art , fun watch beyond fails hold grou..."
104170,zzzjynne,18679,8,"{'Overall': '8', 'Story': '6', 'Animation': '8...",[ contains spoiler ] first half kill la kill f...
126637,zzzjynne,6675,7,"{'Overall': '7', 'Story': '4', 'Animation': '1...","amazing art , fun watch beyond fails hold grou..."
175224,zzzzzzzz,9756,10,"{'Overall': '10', 'Story': '10', 'Animation': ...","madoka hands-down best anime series thus far ,..."


In [88]:
df_review['review_score'].value_counts()

10    42009
9     39695
8     34814
7     25826
6     16497
5     11154
4      7537
3      7204
2      3815
1      3561
Name: review_score, dtype: int64

In [90]:
df = Dataset.load_from_df(df_touse, Reader(rating_scale=(1,10)))
trainset = df.build_full_trainset()

In [93]:
model = SVD()
# model.fit(trainset)
cross_validate(model, df, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.9777  1.9870  1.9841  1.9778  1.9726  1.9798  0.0051  
MAE (testset)     1.5173  1.5291  1.5280  1.5286  1.5239  1.5254  0.0044  
Fit time          5.18    5.15    5.24    5.09    5.21    5.17    0.05    
Test time         0.17    0.15    0.40    0.16    0.16    0.21    0.10    


{'test_rmse': array([1.97768864, 1.98697808, 1.98408158, 1.97781697, 1.97263575]),
 'test_mae': array([1.51726793, 1.52908902, 1.5279525 , 1.52856889, 1.52387204]),
 'fit_time': (5.178173303604126,
  5.152165412902832,
  5.2384209632873535,
  5.088386535644531,
  5.208906412124634),
 'test_time': (0.1690375804901123,
  0.1548471450805664,
  0.3990910053253174,
  0.15703558921813965,
  0.1550440788269043)}

In [96]:
df_touse['profile'].unique()[:10]

array(['skrn', 'Tactician', 'CallMeDette', 'jason200911', 'comiclove',
       'DJM47', 'AnimeBW', 'JohnPerez', 'OVERPOWERED99', 'Anya18'],
      dtype=object)

In [18]:
model.predict('skrn', 28891, 7)

NameError: name 'model' is not defined

In [106]:
df_touse.head(1)

,profile,uid,review_score
0,skrn,28891,7


In [108]:
user_id = 'skrn'
all_anime = df_touse['uid'].unique()
watched = df_touse[df_touse['profile']==user_id].uid
not_watched = [anime for anime in all_anime if anime not in watched]

In [117]:
# predict
score = [model.predict(user_id, anime_id) for anime_id in not_watched]
anime_id = []
pred_score = []
for i in range(0, len(score)):
    anime_id.append(score[i].iid)
    pred_score.append(score[i].est)
df_pred = pd.DataFrame({'uid':anime_id, 'pred_score':pred_score})

In [118]:
df_pred_real = df_pred.sort_values('pred_score', ascending=False).head(10)

In [119]:
df_pred_real.head(1)

,uid,pred_score
133,5114,9.040665


In [120]:
df_pred_real.merge(df_anime, how='left', on='uid')

,uid,pred_score,Unnamed: 0,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,5114,9.040665,3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,"`` order something obtained , something equal ..."
1,28977,8.947396,769,Gintama°,"Gintoki, Shinpachi, and Kagura return as the f...","['Action', 'Comedy', 'Historical', 'Parody', '...","Apr 8, 2015 to Mar 30, 2016",51.0,281594,351,6.0,9.05,"gintoki , shinpachi , kagura return fun-loving..."
2,263,8.852871,761,Hajime no Ippo,Makunouchi Ippo has been bullied his entire li...,"['Comedy', 'Sports', 'Drama', 'Shounen']","Oct 4, 2000 to Mar 27, 2002",75.0,301226,322,29.0,8.77,makunouchi ippo bullied entire life . constant...
3,15335,8.831519,12,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...,When Gintoki apprehends a movie pirate at a pr...,"['Action', 'Sci-Fi', 'Comedy', 'Historical', '...","Jul 6, 2013",1.0,135232,833,14.0,8.95,"gintoki apprehends movie pirate premiere , che..."
4,22135,8.777927,732,Ping Pong the Animation,"""The hero comes. The hero comes. The hero come...","['Drama', 'Psychological', 'Seinen', 'Sports']","Apr 11, 2014 to Jun 20, 2014",11.0,196091,554,59.0,8.62,`` hero come . hero come . hero come . chant w...
5,4181,8.768269,14,Clannad: After Story,"Clannad: After Story , the sequel to the criti...","['Slice of Life', 'Comedy', 'Supernatural', 'D...","Oct 3, 2008 to Mar 27, 2009",24.0,740101,64,12.0,8.97,"clannad : story , sequel critically acclaimed ..."
6,35180,8.755864,766,3-gatsu no Lion 2nd Season,"Now in his second year of high school, Rei Kir...","['Drama', 'Game', 'Seinen', 'Slice of Life']","Oct 14, 2017 to Mar 31, 2018",22.0,169544,657,9.0,9.02,"second year high school , rei kiriyama continu..."
7,9969,8.725731,768,Gintama',"After a one-year hiatus, Shinpachi Shimura ret...","['Action', 'Sci-Fi', 'Comedy', 'Historical', '...","Apr 4, 2011 to Mar 26, 2012",51.0,278110,353,7.0,9.04,"one-year hiatus , shinpachi shimura return edo..."
8,7311,8.717609,739,Suzumiya Haruhi no Shoushitsu,"One cold Christmas day, Kyon heads over to sch...","['Comedy', 'Mystery', 'Romance', 'School', 'Sc...","Feb 6, 2010",1.0,400446,211,51.0,8.68,"one cold christmas day , kyon head school so b..."
9,20583,8.669278,735,Haikyuu!!,Inspired after watching a volleyball ace nickn...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Apr 6, 2014 to Sep 21, 2014",25.0,684654,75,56.0,8.62,inspired watching volleyball ace nicknamed `` ...
